
# 01 Práce s databází pomocí ADO.NET

**autor: Erik Král ekral@utb.cz**

---

V tomto materiálu si probereme práci s databází pomocí knihovny ADO.NET. Všechny probírané příkazy mají i asynchronní verzi, ale pro zjednodušení kód uvádíme synchronní příkazy.

Nejprve si nainstalujeme nuget balíček [Microsoft.Data.Sqlite](https://www.nuget.org/packages/Microsoft.Data.SQLite/) představující database providera pro souborovou databázi *Sqlite*. Tento balíček používají následující příklady.

In [ ]:
#r "nuget: Microsoft.Data.Sqlite"

Installed Packages Microsoft.Data.Sqlite, 9.0.2

## Připojení k databázi

Dále si vytvoříme connection string tak, aby se soubor s databází ukládal do dokumentů uživatele. 

In [ ]:
using Microsoft.Data.Sqlite;
using System.IO;

var folder = Environment.SpecialFolder.MyDocuments;
string folderPath = Environment.GetFolderPath(folder);
string filePath = Path.Join(folderPath, "skola.db");

SqliteConnectionStringBuilder csb = new SqliteConnectionStringBuilder
{
    DataSource = filePath
};

string connectionString = csb.ConnectionString;

Console.WriteLine(connectionString);


Data Source=C:\Users\erik\Documents\skola.db


K databázi se připojíme pomocí následujícího příkazu. Pokud Sqlite databáze ještě neexistuje, tak se vytvoří nová (nový soubor). Třída *SqliteConnection* implemetuje rozhraní *IDispose* a proto se nám připojení uzavře automaticky s využití Dispose patternu a nemusíme jej už ručně uzavírat. 

In [ ]:
SqliteConnection connection;

using(connection = new SqliteConnection(connectionString))
{
    connection.Open();

    Console.WriteLine(connection.State);
}

Console.WriteLine(connection.State);

Open
Closed


## Zadávání dat pomocí ExecuteNonQuery

Pro **vytvoření tabulky**, **vložení nového řádku**, nebo **aktualizaci řádku**, tedy příkazy, které neprovádějí dotaz a nevrací hodnoty používáme metodu ```command.ExecuteNonQuery()```. 

### Vytvoření tabulky

V následujícím kódu vytvoříme novou tabulku `Studenti`, která má sloupce `StudentId`, `Jmeno` a dosažené `Kredity`:

In [ ]:
using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    connection.Open();

    using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            CREATE TABLE Studenti 
            (
                StudentId INTEGER PRIMARY KEY, 
                Jmeno DOUBLE VARCHAR(50),
                Kredity INTEGER
            )
        ";

        int count = command.ExecuteNonQuery();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

0


### Vložení nového řádku

Nyní do vytvořené tabulky vložíme nový řádek. V příkladu používáme parametry commandu `@Jmeno` a `@Kredity`. Hodnoty parametrů můžeme zadávat buď s generickými SQL typy. Nebo s konkrétními typy pro danou databází s metodou, kterou poskytuje konkrétní provider tak jak je ukázáno v příkladu. 

Použití parametrů proskytuje kontrolu typu a validaci zadané hodnoty parametru a pomáhá zabránit útoku technikou SQL Injection. Použití parametrů nezpomaluje provedení dotazu, spíše nám může pomoct dotaz lépe sestavit dle konkrétního typu a díky tomu by provedení dotazu by mohlo být v některých případech efektivnější.

In [ ]:
using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    connection.Open();

    using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            INSERT INTO Studenti (Jmeno, Kredity)
            VALUES (@Jmeno, @Kredity)
        ";

        command.Parameters.Add("@Jmeno", SqliteType.Text).Value = "Karel";
        command.Parameters.Add("@Kredity", SqliteType.Integer).Value = 40;

        int count = command.ExecuteNonQuery();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

Počet změněných řádků: 1


### Změna řádku

Hodnoty řádku změníme obdobým způsobem jako přidávání nových. Konkrétně změníme počet kreditů studenta se `StudentId` 1.

In [ ]:
using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    connection.Open();

    using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =
        @"
            UPDATE Studenti
            SET Kredity = @Kredity 
            WHERE StudentId = @StudentId
        ";

        command.Parameters.Add("@Kredity", SqliteType.Integer).Value = 64;
        command.Parameters.Add("@StudentId", SqliteType.Integer).Value = 1;

        int count = command.ExecuteNonQuery();

        Console.WriteLine($"Počet změněných řádků: {count}");
    }
}

Počet změněných řádků: 1


## Načtení dat pomocí ExecuteReader

Pokud chceme provést dotaz na data tabulky a načíst jednotlivé řádky a sloupce, tak použijeme `reader` jak je uvedeno v následujícím kódu. 

In [ ]:
using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    connection.Open();

    using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText = "SELECT StudentId, Jmeno, Kredity FROM Studenti";

        SqliteDataReader reader = command.ExecuteReader();

        if(reader.HasRows)
        {
            while(reader.Read())
            {
            
                int studentId = reader.GetInt32(reader.GetOrdinal("StudentId"));
                string jmeno = reader.GetString(reader.GetOrdinal("Jmeno"));
                int kredity = reader.GetInt32(reader.GetOrdinal("Kredity"));

                Console.WriteLine($"{studentId}: {jmeno} {kredity}");
            }
        }
    }
}

1: Karel 64
2: Karel 40


## Načtení jedné hodnoty pomocí ExecuteScalar

Prokud provedeme SQL dotaz, který vrátí pouze jednu hodnotu, například ```SELECT AVG(Kredit) FROM Studenti```, tak můžeme použít ```reader``` podobně jako v minulém příkladu a načíst první sloupec prvního řádku. A nebo můžeme použít metodu ```command.ExecuteScalar()``` tak jak je ukázané v následujícím příkladu.

In [ ]:
using(SqliteConnection connection = new SqliteConnection(connectionString))
{
    connection.Open();

    using(SqliteCommand command = connection.CreateCommand())
    {
        command.CommandText =@$"SELECT AVG(Kredity) FROM Studenti";

        object? result = command.ExecuteScalar();

        if(result is double average)
        {
            return average;
        }
    }
}

52

TODO: přidat tabulku Skupiny, cizi klic a relace, ukázka join